<a href="https://colab.research.google.com/github/MuhammadHelmyOmar/Authorship-Identification/blob/main/topic_anlalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Loading Data

In [5]:
from google.colab import drive
import pandas as pd
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
file_path = '/content/drive/MyDrive/Colab Notebooks/AraGenEval/train_data_splitted.csv'

train_data = pd.read_csv(file_path)
display(train_data.head())

,original_id,text_in_author_style,author,text_size,chunk_index,is_split,cleaned_text
0,5843,"هذه الكتب التي أصدرتُها منذ بدأت كتابة باب ""من...",يوسف إدريس,1921,0,False,"هذه الكتب التي أصدرتها منذ بدأت كتابة باب ""من ..."
1,5844,صعب جِدًّا في ظل هذا التقسيم الإرهابي أن أقول ...,يوسف إدريس,1616,0,False,صعب جدا في ظل هذا التقسيم الإرهابي أن أقول إن ...
2,5845,كان ممكنًا أن أتفادى هذا\n\nالموضوع الشائك، وأ...,يوسف إدريس,1913,0,False,كان ممكنا أن أتفادى هذا\n\nالموضوع الشائك، وأذ...
3,5846,ومن المُمكن للعراق وحده أن يَجتثَّه ويقضي على ...,يوسف إدريس,2047,0,False,ومن الممكن للعراق وحده أن يجتثه ويقضي على الهج...
4,5847,كنتُ أريد أن أتناول موضوعًا من هذه المواضيع.\n...,يوسف إدريس,2022,0,False,كنت أريد أن أتناول موضوعا من هذه المواضيع.\n\n...


# Cleaning

In [27]:
import re
import random
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt') # To include and use word tokenizer
nltk.download('punkt_tab') # Download punkt_tab for Arabic tokenization
nltk.download('stopwords')
stop = set(nltk.corpus.stopwords.words("arabic"))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [33]:
def normalize_text(text:str):
  # normalize alif
  text = text.replace(u"\u0625", u"\u0627")  # HAMZA below, with LETTER ALEF
  text = text.replace(u"\u0622", u"\u0627")  # ALEF WITH MADDA ABOVE, with LETTER ALEF
  text = text.replace(u"\u0623", u"\u0627")  # ALEF WITH HAMZA ABOVE, with LETTER ALEF

  # check whether text is of type string
  if not isinstance(text, str):
    raise TypeError("TypeError: text must be a string")

  # remove numbers
  text = re.sub(r"\d+", "رقم", text)

  # Remove punctuations
  text = re.sub(r"[^\w\s]", "", text)

  # remove extra spaces
  text = re.sub(r"\s+", " ", text)

  # This part to make sure of good cleaning (may take long execution time)
  words = word_tokenize(text)
  text = [w for w in words if len(w)>1] # remove non words
  text = " ".join([w for w in words if w not in stop]) # remove stop words

  return text.strip()

In [34]:
train_data['normalized_text'] = train_data['cleaned_text'].apply(normalize_text)
train_data.head()

,original_id,text_in_author_style,author,text_size,chunk_index,is_split,cleaned_text,normalized_text
0,5843,"هذه الكتب التي أصدرتُها منذ بدأت كتابة باب ""من...",يوسف إدريس,1921,0,False,"هذه الكتب التي أصدرتها منذ بدأت كتابة باب ""من ...",الكتب اصدرتها بدات كتابة باب مفكرة يوسف ادريس ...
1,5844,صعب جِدًّا في ظل هذا التقسيم الإرهابي أن أقول ...,يوسف إدريس,1616,0,False,صعب جدا في ظل هذا التقسيم الإرهابي أن أقول إن ...,صعب جدا ظل التقسيم الارهابي ان اقول ان نوعا خا...
2,5845,كان ممكنًا أن أتفادى هذا\n\nالموضوع الشائك، وأ...,يوسف إدريس,1913,0,False,كان ممكنا أن أتفادى هذا\n\nالموضوع الشائك، وأذ...,ممكنا ان اتفادى الموضوع الشائك واذهب اتحدث موض...
3,5846,ومن المُمكن للعراق وحده أن يَجتثَّه ويقضي على ...,يوسف إدريس,2047,0,False,ومن الممكن للعراق وحده أن يجتثه ويقضي على الهج...,الممكن للعراق وحده ان يجتثه ويقضي الهجوم اين م...
4,5847,كنتُ أريد أن أتناول موضوعًا من هذه المواضيع.\n...,يوسف إدريس,2022,0,False,كنت أريد أن أتناول موضوعا من هذه المواضيع.\n\n...,كنت اريد ان اتناول موضوعا المواضيع ولكني استطع...


# Topic Modeling

In [1]:
!pip install -qU bertopic

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.6/150.6 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 115.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 67.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 101.7 MB/s eta 0:00:00


## Embed Articles

In [30]:
from sentence_transformers import SentenceTransformer
import pickle

In [31]:
model_id = "Omartificial-Intelligence-Space/GATE-AraBert-v1"
embedding_model = SentenceTransformer(model_id, device="cuda:0")

In [35]:
articles_embeddings = embedding_model.encode(train_data['normalized_text'].values, show_progress_bar=True, batch_size=64)

print(articles_embeddings.shape)

Batches:   0%|          | 0/515 [00:00<?, ?it/s]

(32941, 768)


In [36]:
# Store data and embeddings
# with open("/content/drive/MyDrive/Colab Notebooks/AraGenEval/authorship_embeddings_GATE-AraBert-v1.pkl", "wb") as fOut:
#   pickle.dump({'embeddings': articles_embeddings},
#               fOut,
#               protocol = pickle.HIGHEST_PROTOCOL)

In [37]:
# Load data and embeddings
with open("/content/drive/MyDrive/Colab Notebooks/AraGenEval/authorship_embeddings_GATE-AraBert-v1.pkl", 'rb') as fIn:
  articles_embeddings = pickle.load(fIn)['embeddings']

## Dimensionality Reduction

In [12]:
from umap import UMAP

In [13]:
umap_model = UMAP(
    n_neighbors = 7,
    n_components = 45,
    min_dist = 0,
    metric = "cosine",
    random_state = 5
)

## Clustering

In [14]:
from hdbscan import HDBSCAN

In [15]:
hdbscan_model = HDBSCAN(
    min_cluster_size = 3,
    metric = "euclidean",
    cluster_selection_method = "eom",
    prediction_data = True
)
# A high min_cluster_size will generate fewer topics
# A low min_cluster_size will generate more topics
# Prediction_data=True -> allows for predicting the cluster membership of new data points based on the trained model
# HDBSCAN can be used to tune outliers

## Vectorizer

In [ ]:
!pip install -qU Arabic-Stopwords

In [17]:
from sklearn.feature_extraction.text import CountVectorizer
import arabicstopwords.arabicstopwords as stp

# Gemi